In [55]:
import os, sys, json
import csv
import string
from tqdm import tqdm
import re
from collections import deque
from difflib import SequenceMatcher
from bs4 import BeautifulSoup
import spacy

import codecs

import glob
import os
import pandas as pd
import numpy as np

In [56]:
# import glob

# txtfiles = []
# for file in glob.glob("*.txt"):
#     txtfiles.append(file)
# txtfiles

In [57]:
# import pandas as pd
# dict = {'name': txtfiles, 'id': txtfiles}
# df = pd.DataFrame(dict)
# df.to_csv('storedScript.csv')

In [88]:
TAGGED_FOLDER = '/Users/yisisang/Desktop/Character/imsdb_copy'
file_list = glob.glob(os.path.join(os.getcwd(), TAGGED_FOLDER, "*.txt"))

corpus = []
files = []

for file_path in file_list:
    with open(file_path, encoding="latin-1") as f_input:
        corpus.append(f_input.read())
        files.append(''.join([n for n in os.path.basename(file_path)]))

df = pd.DataFrame({'file':files, 'text':corpus})

In [90]:
df.head(3)

,file,text
0,Midnight-Express.txt,<pre>\n\n\n<b> ...
1,Big-Eyes.txt,<pre> \n \n\n\n<b> ...
2,Warrior.txt,<pre>\n\n\n<b> ...


In [91]:
df['text2'] = df['text'].str.replace(r'<[^>]*>', '', regex=True).str.strip()
df['story_word_count'] = df['text2'].str.split().str.len()
df.head()

,file,text,text2,story_word_count
0,Midnight-Express.txt,<pre>\n\n\n<b> ...,MIDNIGHT EXPRESS\n\n ...,27409
1,Big-Eyes.txt,<pre> \n \n\n\n<b> ...,BIG EYES\n\n \n\n ...,22624
2,Warrior.txt,<pre>\n\n\n<b> ...,WARRIOR\n \n ...,23320
3,Hellraiser-Hellseeker.txt,<pre>\n\n<b> HELLRAISER: HELLSE...,HELLRAISER: HELLSEEKER\n\n ...,21075
4,Hannah-and-Her-Sisters.txt,<pre><html>\n<head>\n<script>\n<b><!--\n</b>\n...,/*\nBreak-out-of-frames script\nBy Website Abs...,38868


In [92]:
df['file'] = df['file'].str.replace(r'.txt', '')
df.head()

<ipython-input-92-a2c617ad931d>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['file'] = df['file'].str.replace(r'.txt', '')


,file,text,text2,story_word_count
0,Midnight-Express,<pre>\n\n\n<b> ...,MIDNIGHT EXPRESS\n\n ...,27409
1,Big-Eyes,<pre> \n \n\n\n<b> ...,BIG EYES\n\n \n\n ...,22624
2,Warrior,<pre>\n\n\n<b> ...,WARRIOR\n \n ...,23320
3,Hellraiser-Hellseeker,<pre>\n\n<b> HELLRAISER: HELLSE...,HELLRAISER: HELLSEEKER\n\n ...,21075
4,Hannah-and-Her-Sisters,<pre><html>\n<head>\n<script>\n<b><!--\n</b>\n...,/*\nBreak-out-of-frames script\nBy Website Abs...,38868


In [93]:
df = df.rename(columns={'file': 'document_id'})

In [94]:
df = df[['document_id','text','story_word_count']]

In [95]:
df.set_index("document_id", drop=True, inplace=True)
STORY_META = df.to_dict(orient="index")

In [96]:
print(len(STORY_META))

1137


In [100]:
#############################################
##### getting split meta-info via stats #####
#############################################

count_movie = 0
count_meta_complete = 0
count_start_pattern = 0
count_summary = 0
if not os.path.exists('./imsdb_copy/'):
    os.mkdir('./imsdb_copy/')
# if not os.path.exists('./preprocessed/movies/'):
#     os.mkdir('./preprocessed/movies/')
    
count_with_fade_in = 0
list_num_scenes = []
list_num_dialogs = []

num_verified = 0
num_failed = 0
num_movie = 0
num_fade_in_success = 0

meta_fileout = open('imsdb_copy/split_meta_info.txt', 'w')
    
for each_id in STORY_META.keys():
    if each_id in exclusive_id:
        print(each_id, 'has some issues and will be skipped')
        continue
        
    if each_id in special_id:
        print(each_id, 'currently skipping the special script formats')
        continue
#     each_id = 'cd13eb380843224f66c34047cc06dc445a92f8fd'
#     each_id = '18bcdf16fb896faf7fb36db6b752f16f0343ad24'
#     each_id = '1508452704b5829941b98aeee2eeba192f5beae5'
#     each_id = '08373775c3d82d8bbf4ff286a3a0076a3627c652'
    print(each_id)
    num_movie += 1
    
    num_scenes = 0
    num_dialogs = 0
    
    with open(os.path.join('/Users/yisisang/Desktop/Character/imsdb_copy', each_id + '.txt'), 'r', encoding='cp1256') as file:
        content = file.read()

    soup = BeautifulSoup(content, 'html.parser')
    
    bs = soup.pre.find_all('b')

#     result = [[element.string, element.next_sibling.string] for element in bs if (element.next_sibling and element.next_sibling.string is not None)]

    sents = [(element.string, [ele for ele in element.next_sibling.string.split('\n') if ele.strip()]) #split the text by \n; remove space
             for element in bs if (element.next_sibling and element.next_sibling.string is not None)] #select text 
    
    scene_symbol = None
    dialogue_symbol = None
    scene_found = False
    fade_in_indent = -1

    for element in bs:
#         print(element.string)
        if element is None or element.string is None:
            continue
        if not scene_found and element.string.find('FADE IN') != -1:#find if there is a FADE IN in the sentence
            scene_symbol = str(element.string)[:element.string.index('FADE IN')]#count the space before FADE IN
            scene_found = True#only need the 1st FADE IN (incase there is any FADE IN in the text)

        if scene_found and element.string.startswith(scene_symbol + scene_symbol):#find if there are texts start with 2* the space before FADE IN
            dialogue_symbol = scene_symbol + scene_symbol
            break

    if scene_symbol is None:# there is no FADE IN in the script
        print("didn't find the symbol for scenes")
    else:
        print([scene_symbol])
        
        pointer = 0
        if pointer != len(scene_symbol):
            while scene_symbol[pointer] == ' ' or scene_symbol[pointer] == '\t' or scene_symbol[pointer] == '\n':
                pointer += 1
                if pointer == len(scene_symbol):
                    break
        fade_in_indent = pointer
        print('FADE IN indent:', fade_in_indent)
#         fade_in_indent = len(scene_symbol)
        count_with_fade_in += 1
    
    meta_info = {}
    meta_info['id'] = each_id
    if scene_symbol is None:
        meta_info['fade-in symbol'] = []
    else:
        meta_info['fade-in symbol'] = [scene_symbol]
    meta_info['fade-in indent'] = fade_in_indent
    
#     meta_fileout.write('id\t%s\n'%each_id)
#     if scene_symbol is None:
#         meta_fileout.write('%s\n'%json.dumps([]))
#     else:
#         meta_fileout.write('%s\n'%json.dumps([scene_symbol]))
#     meta_fileout.write('fade-in indent\t%d\n'%fade_in_indent)
        
    scene_indent_dict = {}
    indent_dict = {}
    dialog_indent_dict = {}
    section_structures = []
    
    new_sections = []
#     num_movie += 1

    for section in sents:
#         section_name = re.sub(r'\t', '', section[0]).strip()
#     #     section_name = re.sub(r'\t', '', section[0]).strip()
#         section_texts = ''

        prev_indent = -1
        new_scene = False
        
        for sent in section[1]:
            pointer = 0
            if pointer == len(sent):
    #             section_structures.append([section_name, '', ''])
                continue
            while sent[pointer] == ' ' or sent[pointer] == '\t' or sent[pointer] == '\n':
                pointer += 1
                if pointer == len(sent):
                    break

            if pointer == len(sent):
    #             section_structures.append([section_name, '', ''])
                continue

            indent = pointer
    #         if indent > 30:
    #             print(section)
    #         section_type = indent
    #         section_content = section_content[pointer:]
    
#             if fade_in_indent == -1:
            if indent not in indent_dict:
                indent_dict[indent] = 1
            else:
                indent_dict[indent] += 1
    
            if fade_in_indent != -1:
                if indent > fade_in_indent:
                    if indent not in dialog_indent_dict:
                        dialog_indent_dict[indent] = 1
                    else:
                        dialog_indent_dict[indent] += 1
                    num_dialogs += 1
                else:
                    if indent not in scene_indent_dict:
                        scene_indent_dict[indent] = 1
                    else:
                        scene_indent_dict[indent] += 1
                    num_scenes += 1
                    
#                 if indent <= fade_in_indent and prev_indent > fade_in_indent:
#                     new_sections.append(('DIALOG', section_name, section_texts))
#                     num_dialogs += 1
#                     section_name = 'NULL'
#                     section_texts = ''
#                     new_scene = True

#         #         print(sent)
#                 section_texts += re.sub(r'\t', '', sent).strip() + '\n'
#                 prev_indent = indent
               
#         if fade_in_indent != -1:
#             if new_scene:
#                 new_sections.append(('SCENE', section_name, section_texts))
#                 num_scenes += 1
#             else:
#                 if prev_indent > fade_in_indent:
#                     new_sections.append(('DIALOG', section_name, section_texts))
#                     num_dialogs += 1
#                 else:
#                     new_sections.append(('SCENE', section_name, section_texts))
#                     num_scenes += 1
    if len(indent_dict) == 0:
        print('Empty indent dict', indent_dict)
        meta_info['type'] = 'Fail'
        meta_info['indent_dict'] = indent_dict
#         meta_fileout.write('Fail\n')
#         meta_fileout.write(json.dumps(indent_dict) + '\n\n')
    elif fade_in_indent == -1:
        print('OVERALL indents:', indent_dict)
        
        sorted_indents = sorted(indent_dict.items(), key=lambda x:x[0])
        num_total = 0
        for k, v in indent_dict.items():
            num_total += v
            
        scene_indent_dict = {}
        dialog_indent_dict = {}
        cur_num_scenes = 0
        adding_scene = True
        failure = False
        split_num = 0
        for idx, indent_pair in enumerate(sorted_indents):
            cur_ratio = cur_num_scenes / num_total
            next_ratio = (cur_num_scenes + indent_pair[1]) / num_total
            
            if adding_scene:
                print(cur_ratio, next_ratio)
            
            # a huge jump refers to failure case
            if cur_ratio <= 0.3870662796573337 - 0.13559559099176607 and next_ratio >= 0.60: # 0.3870662796573337 + 0.13559559099176607:
                failure = True
                break
            
            if adding_scene and next_ratio >= 0.6: #0.3870662796573337 + 0.13559559099176607: # split here
                adding_scene = False
                split_num = indent_pair[0]
                print('split here')
            if adding_scene:
                scene_indent_dict[indent_pair[0]] = indent_pair[1]
                cur_num_scenes += indent_pair[1]
            else:
                dialog_indent_dict[indent_pair[0]] = indent_pair[1]
        if not failure:
            print('!!!Re-organized overall dicts:')
            print('SCENE indents:', scene_indent_dict)
            print('DIALOG indents:', dialog_indent_dict)
            print('Verified via ratio', 'split at', split_num)
            num_verified += 1
#             meta_fileout.write('Reorg overall\n')
#             meta_fileout.write(json.dumps(indent_dict) + '\n\n')
            meta_info['type'] = 'Reorg overall'
            meta_info['indent_dict'] = indent_dict
            meta_info['scene_dict'] = scene_indent_dict
            meta_info['dialog_dict'] = dialog_indent_dict
        else:
            print('!!!Failed to split (with overall)')
            num_failed += 1
#             meta_fileout.write('Fail\n')
#             meta_fileout.write(json.dumps(indent_dict) + '\n\n')
            meta_info['type'] = 'Fail'
            meta_info['indent_dict'] = indent_dict
        
    else:
        print('SCENE indents:', scene_indent_dict)
        print('DIALOG indents:', dialog_indent_dict)
        list_num_scenes.append(num_scenes)
        list_num_dialogs.append(num_dialogs)
        
        ratio_scene = num_scenes / (num_scenes + num_dialogs)
        if ratio_scene <= 0.6 and ratio_scene >= 0.3870662796573337 - 0.13559559099176607:
            print('!!!Original dicts:')
            print('Verified via ratio')
            num_verified += 1
            num_fade_in_success += 1
#             meta_fileout.write('Original fade-in\n')
#             meta_fileout.write(json.dumps(indent_dict) + '\n\n')
            meta_info['type'] = 'Original fade-in'
            meta_info['indent_dict'] = indent_dict
            meta_info['scene_dict'] = scene_indent_dict
            meta_info['dialog_dict'] = dialog_indent_dict
        else:
            sorted_indents = sorted(indent_dict.items(), key=lambda x:x[0])
            num_total = 0
            for k, v in indent_dict.items():
                num_total += v

            scene_indent_dict = {}
            dialog_indent_dict = {}
            cur_num_scenes = 0
            adding_scene = True
            failure = False
            split_num = 0
            for idx, indent_pair in enumerate(sorted_indents):
                cur_ratio = cur_num_scenes / num_total
                next_ratio = (cur_num_scenes + indent_pair[1]) / num_total
                
                if adding_scene:
                    print(cur_ratio, next_ratio)

                # a huge jump refers to failure case
                if cur_ratio <= 0.3870662796573337 - 0.13559559099176607 and next_ratio >= 0.6: # 0.3870662796573337 + 0.13559559099176607:
                    failure = True
                    break

                if adding_scene and next_ratio >= 0.6: # 0.3870662796573337 + 0.13559559099176607: # split here
                    adding_scene = False
                    split_num = indent_pair[0]
                    print('split here')
                if adding_scene:
                    scene_indent_dict[indent_pair[0]] = indent_pair[1]
                    cur_num_scenes += indent_pair[1]
                else:
                    dialog_indent_dict[indent_pair[0]] = indent_pair[1]
            if not failure:
                print('!!!Re-organized dicts:')
                print('SCENE indents:', scene_indent_dict)
                print('DIALOG indents:', dialog_indent_dict)
                print('Verified via ratio', 'split at', split_num)
                num_verified += 1
#                 meta_fileout.write('Reorg fade-in\n')
#                 meta_fileout.write(json.dumps(scene_indent_dict) + '\n\n')
#                 meta_fileout.write(json.dumps(dialog_indent_dict) + '\n\n')
                meta_info['type'] = 'Reorg fade-in'
                meta_info['indent_dict'] = indent_dict
                meta_info['scene_dict'] = scene_indent_dict
                meta_info['dialog_dict'] = dialog_indent_dict
            else:
                print('!!!Failed to split')
                num_failed += 1
#                 meta_fileout.write('Fail\n')
#                 meta_fileout.write(json.dumps(indent_dict) + '\n\n')
                meta_info['type'] = 'Fail'
                meta_info['indent_dict'] = indent_dict

    meta_fileout.write(json.dumps(meta_info) + '\n')
#     break
meta_fileout.close()


Midnight-Express
['               ']
FADE IN indent: 15
SCENE indents: {15: 2274}
DIALOG indents: {31: 2, 27: 1, 58: 2, 25: 1488, 30: 343, 26: 1, 35: 1}
!!!Original dicts:
Verified via ratio
Big-Eyes
['                         ']
FADE IN indent: 25
SCENE indents: {22: 1, 10: 1348, 11: 1821}
DIALOG indents: {35: 1}
0.0 0.4251024913276569
0.4251024913276569 0.999369284137496
split here
!!!Re-organized dicts:
SCENE indents: {10: 1348}
DIALOG indents: {11: 1821, 22: 1, 35: 1}
Verified via ratio split at 11
Warrior
didn't find the symbol for scenes
OVERALL indents: {31: 1, 13: 1, 10: 1248, 11: 1786, 26: 18}
0.0 0.4086444007858546
0.4086444007858546 0.9934512115258677
split here
!!!Re-organized overall dicts:
SCENE indents: {10: 1248}
DIALOG indents: {11: 1786, 13: 1, 26: 18, 31: 1}
Verified via ratio split at 11
Hellraiser-Hellseeker
didn't find the symbol for scenes
OVERALL indents: {22: 2, 0: 1393, 10: 544, 14: 34, 65: 2, 17: 45, 13: 802, 62: 8, 23: 16, 24: 1, 60: 10, 61: 33, 20: 7, 64: 3

['               ']
FADE IN indent: 15
SCENE indents: {15: 1263}
DIALOG indents: {40: 1, 30: 78, 64: 1, 25: 1367}
!!!Original dicts:
Verified via ratio
Take-Shelter
didn't find the symbol for scenes
OVERALL indents: {34: 1, 33: 1, 10: 1520, 11: 1140}
0.0 0.5709992486851991
0.5709992486851991 0.9992486851990984
split here
!!!Re-organized overall dicts:
SCENE indents: {10: 1520}
DIALOG indents: {11: 1140, 33: 1, 34: 1}
Verified via ratio split at 11
Enough
didn't find the symbol for scenes
OVERALL indents: {35: 1, 33: 1, 10: 2946, 11: 39}
0.0 0.9862738533645798
!!!Failed to split (with overall)
Rocky-Horror-Picture-Show-The
didn't find the symbol for scenes
OVERALL indents: {8: 2, 6: 2, 1: 735, 3: 1333, 5: 54, 4: 31, 0: 49}
0.0 0.022212148685403447
0.022212148685403447 0.35539437896645515
0.35539437896645515 0.9596554850407978
split here
!!!Re-organized overall dicts:
SCENE indents: {0: 49, 1: 735}
DIALOG indents: {3: 1333, 4: 31, 5: 54, 6: 2, 8: 2}
Verified via ratio split at 3
Face-Off

didn't find the symbol for scenes
OVERALL indents: {27: 2, 1: 37, 7: 648, 0: 2002, 9: 12, 5: 7, 10: 3}
0.0 0.7384728882331243
!!!Failed to split (with overall)
Copycat
didn't find the symbol for scenes
Empty indent dict {}
Backdraft
didn't find the symbol for scenes
OVERALL indents: {15: 1612, 25: 2055, 30: 225}
0.0 0.4141829393627955
0.4141829393627955 0.9421891058581706
split here
!!!Re-organized overall dicts:
SCENE indents: {15: 1612}
DIALOG indents: {25: 2055, 30: 225}
Verified via ratio split at 25
Gone-Baby-Gone
didn't find the symbol for scenes
OVERALL indents: {36: 1, 27: 30, 10: 1104, 21: 152, 11: 15, 20: 1555, 25: 50, 26: 137, 24: 32, 28: 3, 30: 3, 23: 95, 15: 2, 19: 4, 32: 1, 22: 10, 31: 2, 33: 21, 35: 1, 75: 1, 16: 6, 71: 1, 37: 1}
0.0 0.3421134180353269
0.3421134180353269 0.34676169817167646
0.34676169817167646 0.3473814688565231
0.3473814688565231 0.3492407809110629
0.3492407809110629 0.3504803222807561
0.3504803222807561 0.8323520297489929
split here
!!!Re-organized ove

['']
FADE IN indent: 0
SCENE indents: {0: 477}
DIALOG indents: {26: 116, 25: 86, 38: 23, 23: 877, 7: 67, 8: 157, 32: 16, 41: 8, 31: 80, 39: 6, 9: 6, 40: 4, 17: 20, 15: 217, 33: 16, 18: 23, 30: 6, 43: 2, 10: 13, 34: 9, 29: 20, 24: 111, 27: 2, 42: 3, 44: 1, 36: 4, 16: 1, 49: 1, 47: 1}
0.0 0.20101137800252844
0.20101137800252844 0.2292456805731142
0.2292456805731142 0.29540665823851664
0.29540665823851664 0.29793510324483774
0.29793510324483774 0.3034134007585335
0.3034134007585335 0.3948588284871471
0.3948588284871471 0.3952802359882006
0.3952802359882006 0.40370838600927095
0.40370838600927095 0.4134007585335019
0.4134007585335019 0.7829751369574378
split here
!!!Re-organized dicts:
SCENE indents: {0: 477, 7: 67, 8: 157, 9: 6, 10: 13, 15: 217, 16: 1, 17: 20, 18: 23}
DIALOG indents: {23: 877, 24: 111, 25: 86, 26: 116, 27: 2, 29: 20, 30: 6, 31: 80, 32: 16, 33: 16, 34: 9, 36: 4, 38: 23, 39: 6, 40: 4, 41: 8, 42: 3, 43: 2, 44: 1, 47: 1, 49: 1}
Verified via ratio split at 23
Barry-Lyndon
['']

didn't find the symbol for scenes
OVERALL indents: {0: 1048, 1: 2111, 2: 176, 3: 10, 7: 2}
0.0 0.31311622348371676
0.31311622348371676 0.9438302957872722
split here
!!!Re-organized overall dicts:
SCENE indents: {0: 1048}
DIALOG indents: {1: 2111, 2: 176, 3: 10, 7: 2}
Verified via ratio split at 1
Brothers-Bloom-The
['     ']
FADE IN indent: 5
SCENE indents: {0: 93, 4: 20, 5: 551}
DIALOG indents: {29: 32, 28: 31, 14: 13, 24: 1, 63: 23, 17: 55, 65: 19, 27: 3, 66: 13, 15: 335, 67: 14, 18: 637, 19: 642, 22: 17, 64: 25, 16: 239, 26: 14, 62: 15, 25: 10, 68: 12, 31: 2, 23: 19, 61: 2, 54: 2, 30: 4, 70: 1, 59: 1, 6: 811, 20: 4, 21: 3, 71: 1, 69: 1}
0.0 0.025375170532060027
0.025375170532060027 0.030832196452933152
0.030832196452933152 0.18117326057298772
0.18117326057298772 0.4024556616643929
0.4024556616643929 0.40600272851296043
0.40600272851296043 0.4974079126875853
0.4974079126875853 0.5626193724420191
0.5626193724420191 0.5776261937244201
0.5776261937244201 0.7514324693042292
split here
!!

...

didn't find the symbol for scenes
OVERALL indents: {15: 1084, 25: 2425, 30: 259}
0.0 0.28768577494692144
0.28768577494692144 0.9312632696390658
split here
!!!Re-organized overall dicts:
SCENE indents: {15: 1084}
DIALOG indents: {25: 2425, 30: 259}
Verified via ratio split at 25
Her
didn't find the symbol for scenes
OVERALL indents: {31: 2, 0: 197, 10: 44, 14: 99, 18: 3, 15: 963, 11: 171, 17: 8, 16: 149, 12: 827, 1: 503, 19: 89, 20: 14, 22: 5, 23: 1, 26: 2, 25: 2, 21: 1, 13: 13, 2: 57}
0.0 0.06253968253968253
0.06253968253968253 0.2222222222222222
0.2222222222222222 0.24031746031746032
0.24031746031746032 0.2542857142857143
0.2542857142857143 0.30857142857142855
0.30857142857142855 0.5711111111111111
0.5711111111111111 0.5752380952380952
0.5752380952380952 0.6066666666666667
split here
!!!Re-organized overall dicts:
SCENE indents: {0: 197, 1: 503, 2: 57, 10: 44, 11: 171, 12: 827, 13: 13}
DIALOG indents: {14: 99, 15: 963, 16: 149, 17: 8, 18: 3, 19: 89, 20: 14, 21: 1, 22: 5, 23: 1, 25: 2,

didn't find the symbol for scenes
OVERALL indents: {38: 2, 37: 1, 12: 993, 22: 2154, 26: 121, 32: 4, 59: 2, 27: 2, 63: 1, 58: 1}
0.0 0.3026516306004267
0.3026516306004267 0.9591587930508991
split here
!!!Re-organized overall dicts:
SCENE indents: {12: 993}
DIALOG indents: {22: 2154, 26: 121, 27: 2, 32: 4, 37: 1, 38: 2, 58: 1, 59: 2, 63: 1}
Verified via ratio split at 22
Master-The
didn't find the symbol for scenes
OVERALL indents: {28: 1, 23: 1, 10: 2897, 11: 10}
0.0 0.9958748710897215
!!!Failed to split (with overall)
Replacements-The
['       ']
FADE IN indent: 7
SCENE indents: {7: 20, 0: 1785, 6: 93, 5: 149, 4: 1, 1: 1, 3: 1}
DIALOG indents: {43: 1, 16: 71, 9: 1686, 12: 347, 10: 99, 20: 16, 17: 81, 18: 20, 19: 34, 11: 329, 14: 17, 21: 1, 13: 25, 22: 1}
!!!Original dicts:
Verified via ratio
Robin-Hood-Prince-of-Thieves
['']
FADE IN indent: 0
SCENE indents: {0: 1928}
DIALOG indents: {25: 7, 22: 2, 11: 42, 54: 35, 23: 71, 9: 1793, 56: 13, 55: 26, 53: 28, 24: 5, 16: 98, 57: 14, 12: 103,

didn't find the symbol for scenes
OVERALL indents: {27: 13, 0: 34, 4: 141, 16: 223, 50: 3, 63: 13, 28: 42, 17: 352, 13: 21, 19: 343, 65: 15, 23: 34, 29: 26, 67: 14, 31: 12, 21: 61, 14: 216, 66: 22, 20: 48, 70: 5, 34: 5, 56: 6, 15: 416, 84: 1, 25: 17, 68: 9, 12: 32, 77: 2, 30: 17, 64: 16, 18: 831, 24: 87, 73: 2, 33: 4, 71: 7, 48: 1, 1: 2, 5: 526, 51: 2, 10: 1, 69: 5, 26: 23, 22: 25, 32: 3, 9: 15, 61: 2, 62: 2, 60: 1, 74: 2, 42: 1, 38: 4, 40: 1, 72: 3, 37: 2, 55: 1, 6: 176, 54: 1, 57: 1, 75: 2, 53: 1, 81: 1, 43: 4}
0.0 0.008722421754746024
0.008722421754746024 0.009235505387378143
0.009235505387378143 0.045407901487942534
0.045407901487942534 0.18034889687018985
0.18034889687018985 0.22550025654181632
0.22550025654181632 0.2293483837865572
0.2293483837865572 0.22960492560287327
0.22960492560287327 0.23781426372498718
0.23781426372498718 0.2432016418676244
0.2432016418676244 0.2986146741918933
0.2986146741918933 0.40533606977937403
0.40533606977937403 0.4625448948178553
0.4625448948178553

['                         ']
FADE IN indent: 25
SCENE indents: {10: 1415, 11: 1479, 25: 91}
DIALOG indents: {33: 1, 29: 1, 31: 1, 26: 15}
0.0 0.47119547119547117
0.47119547119547117 0.9637029637029637
split here
!!!Re-organized dicts:
SCENE indents: {10: 1415}
DIALOG indents: {11: 1479, 25: 91, 26: 15, 29: 1, 31: 1, 33: 1}
Verified via ratio split at 11
Mariachi-El
didn't find the symbol for scenes
OVERALL indents: {0: 981, 61: 20, 10: 496, 14: 92, 62: 6, 15: 5, 65: 1, 13: 9, 20: 2, 58: 7, 59: 6, 16: 3, 12: 5, 60: 14, 17: 6, 52: 1, 9: 24}
0.0 0.5846245530393326
0.5846245530393326 0.598927294398093
0.598927294398093 0.8945172824791419
split here
!!!Re-organized overall dicts:
SCENE indents: {0: 981, 9: 24}
DIALOG indents: {10: 496, 12: 5, 13: 9, 14: 92, 15: 5, 16: 3, 17: 6, 20: 2, 52: 1, 58: 7, 59: 6, 60: 14, 61: 20, 62: 6, 65: 1}
Verified via ratio split at 10
Suspect-Zero
['    ']
FADE IN indent: 4
SCENE indents: {4: 153, 0: 461, 1: 1}
DIALOG indents: {30: 1, 31: 1, 25: 1, 23: 1, 51:

['               ']
FADE IN indent: 15
SCENE indents: {15: 1343}
DIALOG indents: {44: 1, 37: 1, 34: 1, 39: 1, 25: 1416, 30: 247, 36: 2, 38: 1}
!!!Original dicts:
Verified via ratio
Silver-Linings-Playbook
['          ']
FADE IN indent: 10
SCENE indents: {10: 546}
DIALOG indents: {31: 1, 28: 1, 18: 1, 52: 1, 11: 2795}
0.0 0.16322869955156952
0.16322869955156952 0.9988041853512706
!!!Failed to split
Four-Rooms
['               ']
FADE IN indent: 15
SCENE indents: {15: 1244}
DIALOG indents: {25: 2178, 30: 230, 38: 1, 37: 1, 36: 1, 33: 1}
!!!Original dicts:
Verified via ratio


In [101]:
print('%d scripts satisfying the ratio constraints.' % num_verified)
print('\tAmong them there are %d handled with FADE IN only.' % num_fade_in_success)
print('%d scripts failed' % num_failed)
print('%d total scripts' % num_movie)

789 scripts satisfying the ratio constraints.
	Among them there are 303 handled with FADE IN only.
287 scripts failed
1137 total scripts


In [111]:
#############################
##### Splitting scripts #####
#############################

count_movie = 0
count_meta_complete = 0
count_start_pattern = 0
count_summary = 0
if not os.path.exists('./imsdb_copy/silver/'):
    os.mkdir('./imsdb_copy/silver/')
if not os.path.exists('./imsdb_copy/by_stats/'):
    os.mkdir('./imsdb_copy/by_stats/')
# if not os.path.exists('./preprocessed/movies/'):
#     os.mkdir('./preprocessed/movies/')
    
count_with_fade_in = 0
list_num_scenes = []
list_num_dialogs = []

num_verified = 0
num_failed = 0
num_movie = 0
num_fade_in_success = 0

SPLIT_META = {}
meta_filein = open('split_meta_info.txt')
for line in meta_filein:
    meta_info = json.loads(line.strip())
    SPLIT_META[meta_info['id']] = meta_info
    
for each_id in STORY_META.keys():
    if each_id in exclusive_id:
        print(each_id, 'has some issues and will be skipped')
        continue
        
    if each_id in special_id:
        print(each_id, 'currently skipping the special script formats')
        continue
#     each_id = 'cd13eb380843224f66c34047cc06dc445a92f8fd'
#     each_id = '18bcdf16fb896faf7fb36db6b752f16f0343ad24'
#     each_id = '1508452704b5829941b98aeee2eeba192f5beae5'
#     each_id = '00ee9e01a0e581e0d8cbf7e865a895147c480c5e'
    print(each_id)
    if SPLIT_META[each_id]['type'] == 'Fail':
        continue
    elif SPLIT_META[each_id]['type'].startswith('Original'):
        fileout = open(os.path.join('./imsdb_copy/silver', each_id + '.txt'), 'w', encoding='cp1256')
    elif SPLIT_META[each_id]['type'].startswith('Reorg'):
#         continue
        fileout = open(os.path.join('./imsdb_copy/by_stats', each_id + '.txt'), 'w', encoding='cp1256')
        
    print(SPLIT_META[each_id])
        
    num_movie += 1
    
    num_scenes = 0
    num_dialogs = 0
    
    with open(os.path.join('/Users/yisisang/Desktop/Character/imsdb_copy', each_id + '.txt'), 'r', encoding='cp1256') as file:
        content = file.read()

    soup = BeautifulSoup(content, 'html.parser')
    
    bs = soup.pre.find_all('b')

#     result = [[element.string, element.next_sibling.string] for element in bs if (element.next_sibling and element.next_sibling.string is not None)]

    sents = [(element.string, [ele for ele in element.next_sibling.string.split('\n') if ele.strip()]) #split the text by \n; remove space
             for element in bs if (element.next_sibling and element.next_sibling.string is not None)] #select text 
    
    scene_symbol = None
    dialogue_symbol = None
    scene_found = False
    fade_in_indent = -1

    for element in bs:
#         print(element.string)
        if element is None or element.string is None:
            continue
        if not scene_found and element.string.find('FADE IN') != -1:#find if there is a FADE IN in the sentence
            scene_symbol = str(element.string)[:element.string.index('FADE IN')]#count the space before FADE IN
            scene_found = True#only need the 1st FADE IN (incase there is any FADE IN in the text)

        if scene_found and element.string.startswith(scene_symbol + scene_symbol):#find if there are texts start with 2* the space before FADE IN
            dialogue_symbol = scene_symbol + scene_symbol
            break

    if scene_symbol is None:# there is no FADE IN in the script
        print("didn't find the symbol for scenes")
    else:
        print([scene_symbol])
        
        pointer = 0
        if pointer != len(scene_symbol):
            while scene_symbol[pointer] == ' ' or scene_symbol[pointer] == '\t' or scene_symbol[pointer] == '\n':
                pointer += 1
                if pointer == len(scene_symbol):
                    break
        fade_in_indent = pointer
        print('FADE IN indent:', fade_in_indent)
#         fade_in_indent = len(scene_symbol)
        count_with_fade_in += 1
        
    scene_indent_dict = {}
    indent_dict = {}
    dialog_indent_dict = {}
    section_structures = []
    
    new_sections = []
#     num_movie += 1

    for section in sents:
#         print(section)
        if len(section[1]) == 0:
            continue
        if section[0] is not None:
            section_name = re.sub(r'\t', '', section[0]).strip()
        else:
            section_name = 'NULL'
    #     section_name = re.sub(r'\t', '', section[0]).strip()
        section_texts = ''

        prev_indent = -1
        new_scene = False
        
        for sent in section[1]:
            pointer = 0
            if pointer == len(sent):
    #             section_structures.append([section_name, '', ''])
                continue
            while sent[pointer] == ' ' or sent[pointer] == '\t' or sent[pointer] == '\n':
                pointer += 1
                if pointer == len(sent):
                    break

            if pointer == len(sent):
    #             section_structures.append([section_name, '', ''])
                continue

            indent = str(pointer)
#             print(indent)
                    
            if indent in SPLIT_META[each_id]['scene_dict'] and prev_indent in SPLIT_META[each_id]['dialog_dict']:
                new_sections.append(('DIALOG', section_name, section_texts))
                num_dialogs += 1
                section_name = 'NULL'
                section_texts = ''
                new_scene = True

            section_texts += re.sub(r'\t', '', sent).strip() + '\n'
            prev_indent = indent
               
        if new_scene:
            new_sections.append(('SCENE', section_name, section_texts))
            num_scenes += 1
        else:
            if indent in SPLIT_META[each_id]['dialog_dict']:
                new_sections.append(('DIALOG', section_name, section_texts))
                num_dialogs += 1
            elif indent in SPLIT_META[each_id]['scene_dict']:
                new_sections.append(('SCENE', section_name, section_texts))
                num_scenes += 1
            else:
                print('ERROR: unknown indent:', prev_indent)
#                 print(section)

    print('num_scenes:', num_scenes)
    print('num_dialogs:', num_dialogs)
    for new_section in new_sections:
    #     print(new_section)
        fileout.write(json.dumps(new_section))
        fileout.write('\n')
    fileout.close()
#     break


Midnight-Express
{'id': 'Midnight-Express', 'fade-in symbol': ['               '], 'fade-in indent': 15, 'type': 'Original fade-in', 'indent_dict': {'31': 2, '27': 1, '58': 2, '15': 2274, '25': 1488, '30': 343, '26': 1, '35': 1}, 'scene_dict': {'15': 2274}, 'dialog_dict': {'31': 2, '27': 1, '58': 2, '25': 1488, '30': 343, '26': 1, '35': 1}}
['               ']
FADE IN indent: 15
num_scenes: 483
num_dialogs: 640
Big-Eyes
{'id': 'Big-Eyes', 'fade-in symbol': ['                         '], 'fade-in indent': 25, 'type': 'Reorg fade-in', 'indent_dict': {'35': 1, '22': 1, '10': 1348, '11': 1821}, 'scene_dict': {'10': 1348}, 'dialog_dict': {'11': 1821, '22': 1, '35': 1}}
['                         ']
FADE IN indent: 25
num_scenes: 577
num_dialogs: 862
Warrior
{'id': 'Warrior', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'31': 1, '13': 1, '10': 1248, '11': 1786, '26': 18}, 'scene_dict': {'10': 1248}, 'dialog_dict': {'11': 1786, '13': 1, '26': 18, '31': 

['\t\t\t\t']
FADE IN indent: 4
num_scenes: 534
num_dialogs: 859
Bad-Country
{'id': 'Bad-Country', 'fade-in symbol': ['            '], 'fade-in indent': 12, 'type': 'Original fade-in', 'indent_dict': {'31': 1, '34': 1, '30': 1, '26': 197, '50': 1, '12': 1441, '22': 1387, '27': 1}, 'scene_dict': {'12': 1441}, 'dialog_dict': {'31': 1, '34': 1, '30': 1, '26': 197, '50': 1, '22': 1387, '27': 1}}
['            ']
FADE IN indent: 12
num_scenes: 640
num_dialogs: 725
17-Again
{'id': '17-Again', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'37': 1, '36': 1, '35': 2, '57': 1, '10': 1061, '30': 633, '20': 1542, '31': 82, '69': 8, '34': 7, '66': 17, '32': 56, '24': 33, '73': 1, '23': 2, '21': 69, '25': 5, '33': 38, '67': 24, '68': 14, '58': 1, '65': 25, '70': 6, '63': 2, '62': 3, '60': 2, '22': 49, '26': 1, '29': 6, '64': 2, '71': 1, '61': 1, '59': 1}, 'scene_dict': {'10': 1061}, 'dialog_dict': {'20': 1542, '21': 69, '22': 49, '23': 2, '24': 33, '25': 5, '26'

['               ']
FADE IN indent: 15
num_scenes: 436
num_dialogs: 589
Elephant-Man-The
{'id': 'Elephant-Man-The', 'fade-in symbol': ['               '], 'fade-in indent': 15, 'type': 'Original fade-in', 'indent_dict': {'40': 1, '21': 1, '41': 1, '23': 1, '44': 1, '38': 1, '15': 2213, '25': 2206, '30': 157}, 'scene_dict': {'15': 2213}, 'dialog_dict': {'40': 1, '21': 1, '41': 1, '23': 1, '44': 1, '38': 1, '25': 2206, '30': 157}}
['               ']
FADE IN indent: 15
num_scenes: 560
num_dialogs: 905
Speed-Racer
Piano-The
GI-Jane
{'id': 'GI-Jane', 'fade-in symbol': [''], 'fade-in indent': 0, 'type': 'Original fade-in', 'indent_dict': {'29': 1, '24': 1, '44': 1, '0': 1184, '10': 2148, '18': 116, '19': 5, '25': 1}, 'scene_dict': {'0': 1184}, 'dialog_dict': {'29': 1, '24': 1, '44': 1, '10': 2148, '18': 116, '19': 5, '25': 1}}
['']
FADE IN indent: 0
num_scenes: 410
num_dialogs: 845
Bamboozled
{'id': 'Bamboozled', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_d

['          ']
FADE IN indent: 10
num_scenes: 1142
num_dialogs: 2151
John-Wick
Resident-Evil
Great-Gatsby-The
{'id': 'Great-Gatsby-The', 'fade-in symbol': ['          IMAGES OF DECADENT NEW YORK '], 'fade-in indent': 10, 'type': 'Original fade-in', 'indent_dict': {'34': 1, '24': 58, '35': 7, '21': 21, '10': 1150, '20': 2091, '39': 57, '69': 4, '68': 5, '70': 1, '45': 3, '67': 5, '40': 2, '66': 6, '25': 1, '41': 1, '42': 2, '23': 5, '65': 3, '22': 13}, 'scene_dict': {'10': 1150}, 'dialog_dict': {'34': 1, '24': 58, '35': 7, '21': 21, '20': 2091, '39': 57, '69': 4, '68': 5, '70': 1, '45': 3, '67': 5, '40': 2, '66': 6, '25': 1, '41': 1, '42': 2, '23': 5, '65': 3, '22': 13}}
['          IMAGES OF DECADENT NEW YORK ']
FADE IN indent: 10
num_scenes: 660
num_dialogs: 1152
Law-Abiding-Citizen
{'id': 'Law-Abiding-Citizen', 'fade-in symbol': ['    '], 'fade-in indent': 4, 'type': 'Reorg fade-in', 'indent_dict': {'29': 3, '20': 73, '4': 63, '0': 220, '27': 6, '17': 48, '24': 6, '18': 573, '25': 39

['        ']
FADE IN indent: 8
num_scenes: 369
num_dialogs: 536
Ugly-Truth-The
{'id': 'Ugly-Truth-The', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'22': 52, '27': 4, '29': 5, '19': 193, '28': 24, '23': 16, '16': 170, '52': 1, '4': 36, '14': 41, '18': 1115, '66': 7, '11': 109, '17': 102, '21': 7, '67': 5, '12': 3, '5': 644, '65': 75, '15': 686, '68': 3, '25': 10, '13': 1, '62': 1, '61': 4, '20': 18, '63': 2, '31': 1, '69': 1, '10': 1, '26': 2, '64': 5, '6': 139, '30': 1}, 'scene_dict': {'4': 36, '5': 644, '6': 139, '10': 1, '11': 109, '12': 3, '13': 1, '14': 41, '15': 686, '16': 170, '17': 102}, 'dialog_dict': {'18': 1115, '19': 193, '20': 18, '21': 7, '22': 52, '23': 16, '25': 10, '26': 2, '27': 4, '28': 24, '29': 5, '30': 1, '31': 1, '52': 1, '61': 4, '62': 1, '63': 2, '64': 5, '65': 75, '66': 7, '67': 5, '68': 3, '69': 1}}
didn't find the symbol for scenes
num_scenes: 931
num_dialogs: 760
As-Good-As-It-Gets
{'id': 'As-Good-As-It-Gets', 'fade-

...

didn't find the symbol for scenes
num_scenes: 750
num_dialogs: 619
Minis-First-Time
{'id': 'Minis-First-Time', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'30': 348, '53': 17, '10': 1400, '20': 936, '11': 25, '31': 40, '69': 22, '60': 33, '33': 508, '23': 1318, '35': 7, '34': 44, '61': 47, '67': 17, '27': 64, '29': 1, '51': 9, '70': 1, '24': 56, '63': 12, '28': 6, '68': 19, '66': 44, '62': 12, '36': 9, '55': 1, '73': 1, '56': 1, '32': 2, '57': 2, '25': 2, '64': 1, '21': 1, '58': 2, '12': 1, '65': 18}, 'scene_dict': {'10': 1400, '11': 25, '12': 1, '20': 936, '21': 1}, 'dialog_dict': {'23': 1318, '24': 56, '25': 2, '27': 64, '28': 6, '29': 1, '30': 348, '31': 40, '32': 2, '33': 508, '34': 44, '35': 7, '36': 9, '51': 9, '53': 17, '55': 1, '56': 1, '57': 2, '58': 2, '60': 33, '61': 47, '62': 12, '63': 12, '64': 1, '65': 18, '66': 44, '67': 17, '68': 19, '69': 22, '70': 1, '73': 1}}
didn't find the symbol for scenes
num_scenes: 1267
num_dialogs: 1841

num_scenes: 508
num_dialogs: 712
I-am-Sam
{'id': 'I-am-Sam', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'38': 2, '37': 1, '12': 993, '22': 2154, '26': 121, '32': 4, '59': 2, '27': 2, '63': 1, '58': 1}, 'scene_dict': {'12': 993}, 'dialog_dict': {'22': 2154, '26': 121, '27': 2, '32': 4, '37': 1, '38': 2, '58': 1, '59': 2, '63': 1}}
didn't find the symbol for scenes
num_scenes: 403
num_dialogs: 993
Master-The
Replacements-The
{'id': 'Replacements-The', 'fade-in symbol': ['       '], 'fade-in indent': 7, 'type': 'Original fade-in', 'indent_dict': {'43': 1, '7': 20, '0': 1785, '6': 93, '16': 71, '9': 1686, '12': 347, '10': 99, '20': 16, '17': 81, '18': 20, '19': 34, '11': 329, '5': 149, '14': 17, '21': 1, '13': 25, '22': 1, '4': 1, '1': 1, '3': 1}, 'scene_dict': {'7': 20, '0': 1785, '6': 93, '5': 149, '4': 1, '1': 1, '3': 1}, 'dialog_dict': {'43': 1, '16': 71, '9': 1686, '12': 347, '10': 99, '20': 16, '17': 81, '18': 20, '19': 34, '11': 329, '14': 1

['               ']
FADE IN indent: 15
num_scenes: 592
num_dialogs: 917
Road-The
Who-Framed-Roger-Rabbit
Fifth-Element-The
{'id': 'Fifth-Element-The', 'fade-in symbol': [''], 'fade-in indent': 0, 'type': 'Reorg fade-in', 'indent_dict': {'1': 793, '0': 533, '5': 234, '4': 1635, '3': 5, '6': 1}, 'scene_dict': {'0': 533, '1': 793, '3': 5}, 'dialog_dict': {'4': 1635, '5': 234, '6': 1}}
['']
FADE IN indent: 0
num_scenes: 680
num_dialogs: 944
Good-Girl-The
Thor
Devils-Advocate
{'id': 'Devils-Advocate', 'fade-in symbol': ['    '], 'fade-in indent': 4, 'type': 'Reorg fade-in', 'indent_dict': {'24': 29, '15': 143, '19': 992, '22': 23, '4': 79, '62': 3, '17': 282, '64': 9, '14': 50, '21': 6, '25': 79, '59': 1, '18': 812, '26': 116, '5': 312, '63': 7, '68': 3, '27': 36, '66': 8, '65': 3, '6': 524, '61': 2, '0': 27, '16': 231, '23': 17, '7': 170, '20': 113, '29': 1, '67': 3, '28': 2}, 'scene_dict': {'0': 27, '4': 79, '5': 312, '6': 524, '7': 170, '14': 50, '15': 143, '16': 231, '17': 282}, 'dialog

['               ']
FADE IN indent: 15
num_scenes: 647
num_dialogs: 1221
What-Lies-Beneath
{'id': 'What-Lies-Beneath', 'fade-in symbol': [''], 'fade-in indent': 0, 'type': 'Original fade-in', 'indent_dict': {'30': 1, '25': 1, '48': 1, '0': 1263, '10': 1398, '15': 149, '3': 2, '16': 5, '21': 1}, 'scene_dict': {'0': 1263}, 'dialog_dict': {'30': 1, '25': 1, '48': 1, '10': 1398, '15': 149, '3': 2, '16': 5, '21': 1}}
['']
FADE IN indent: 0
num_scenes: 565
num_dialogs: 926
Two-For-The-Money
Clash-of-the-Titans
Light-Sleeper
{'id': 'Light-Sleeper', 'fade-in symbol': [], 'fade-in indent': -1, 'type': 'Reorg overall', 'indent_dict': {'40': 1, '38': 1, '15': 666, '30': 325, '25': 1463}, 'scene_dict': {'15': 666}, 'dialog_dict': {'25': 1463, '30': 325, '38': 1, '40': 1}}
didn't find the symbol for scenes
num_scenes: 179
num_dialogs: 773
Dog-Day-Afternoon
{'id': 'Dog-Day-Afternoon', 'fade-in symbol': ['               '], 'fade-in indent': 15, 'type': 'Original fade-in', 'indent_dict': {'44': 1, '3

['                                                      ']
FADE IN indent: 54
num_scenes: 758
num_dialogs: 599
Wild-Wild-West
{'id': 'Wild-Wild-West', 'fade-in symbol': ['     '], 'fade-in indent': 5, 'type': 'Reorg fade-in', 'indent_dict': {'28': 7, '19': 536, '24': 10, '17': 72, '13': 1, '16': 109, '23': 16, '40': 1, '0': 13, '5': 976, '32': 19, '25': 89, '26': 90, '18': 945, '4': 37, '21': 3, '14': 8, '15': 77, '22': 10, '62': 16, '29': 4, '33': 2, '65': 3, '35': 1, '63': 36, '61': 6, '66': 3, '27': 32, '36': 2, '60': 1, '64': 10, '59': 2, '7': 120, '6': 851, '20': 86, '47': 1, '43': 1, '68': 1, '67': 1}, 'scene_dict': {'0': 13, '4': 37, '5': 976, '6': 851, '7': 120, '13': 1, '14': 8, '15': 77, '16': 109, '17': 72}, 'dialog_dict': {'18': 945, '19': 536, '20': 86, '21': 3, '22': 10, '23': 16, '24': 10, '25': 89, '26': 90, '27': 32, '28': 7, '29': 4, '32': 19, '33': 2, '35': 1, '36': 2, '40': 1, '43': 1, '47': 1, '59': 2, '60': 1, '61': 6, '62': 16, '63': 36, '64': 10, '65': 3, '66': 

In [112]:
print(count_with_fade_in)
print(list_num_scenes)
print(list_num_dialogs)

list_ratio_scenes = []
list_ratio_dialogs = []

for i, (x, y) in enumerate(zip(list_num_scenes, list_num_dialogs)):
    if x / (x+y) > 0.9 or x / (x+y) < 0.1:
        continue # skip the extreme cases
    list_ratio_scenes.append(x / (x+y))
    list_ratio_dialogs.append(y / (x+y))
print(len(list_ratio_scenes))

430
[]
[]
0


In [113]:
# print(list_ratio_scenes)
# print(list_ratio_dialogs)

import numpy as np
print('mean num of scenes:', np.mean(list_ratio_scenes))
print('std num of scenes:', np.std(list_ratio_scenes))
print('mean num of dialogs:', np.mean(list_ratio_dialogs))
print('std num of dialogs:', np.std(list_ratio_dialogs))

mean num of scenes: nan
std num of scenes: nan
mean num of dialogs: nan
std num of dialogs: nan


/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:

with open(os.path.join('tmp', 'ff53fd53a94f343b8365915645b79d7ad5b1528e.content'), 'r') as file:
    content = file.read()
    
soup = BeautifulSoup(content, 'html.parser')

#print(soup.prettify())
soup.pre


In [ ]:
# bs = soup.find_all('b')
bs = soup.pre.find_all('b')

result = [[element.string, element.next_sibling.string] for element in bs if element.next_sibling]

sents = [(element.string, [ele for ele in element.next_sibling.string.split('\n') if ele.strip()]) #split the text by \n; remove space
         for element in bs if element.next_sibling] #select text 

for sent in sents:
#     if 'Writers' in sent:
    print(sent)

In [114]:
#FADE IN align at scene；the space before dialogue >= 2* the space before FADE IN
scene_symbol = None
dialogue_symbol = None
scene_found = False

for element in bs:
    if not scene_found and element.string.find('FADE IN') != -1:#find if there is a FADE IN in the sentence
        scene_symbol = str(element.string)[:element.string.index('FADE IN')]#count the space before FADE IN
        scene_found = True#only need the 1st FADE IN (incase there is any FADE IN in the text)

    if scene_found and element.string.startswith(scene_symbol + scene_symbol):#find if there are texts start with 2* the space before FADE IN
        dialogue_symbol = scene_symbol + scene_symbol
        break

if scene_symbol is None:# there is no FADE IN in the script
    print("didn't find the symbol for scenes")
else:
    print([scene_symbol])
    print('FADE IN indent:', len(scene_symbol))
    fade_in_indent = len(scene_symbol)

# if scene_symbol is None:# there is no FADE IN in the script
#     print("didn't find the symbol for scenes")
# else:
#     print([scene_symbol])

# if dialogue_symbol is None:#the space before dialogue != 2* the space before FADE IN
#     print("didn't find the symbol for dialogue")
# else:
#     print([dialogue_symbol])

indent_dict = {}
section_structures = []

new_sections = []
num_scenes = 0
num_dialogs = 0

for section in sents:
    section_name = re.sub(r'\t', '', section[0]).strip()
#     section_name = re.sub(r'\t', '', section[0]).strip()
    section_texts = ''
#     print(section_name)
    
    prev_indent = -1
    new_scene = False
    for (sent_id, sent) in enumerate(section[1]):
        pointer = 0
        if pointer == len(sent):
#             section_structures.append([section_name, '', ''])
            continue
        while sent[pointer] == ' ' or sent[pointer] == '\t' or sent[pointer] == '\n':
            pointer += 1
            if pointer == len(sent):
                break
                
        if pointer == len(sent):
#             section_structures.append([section_name, '', ''])
            continue

        indent = pointer
#         if indent > 30:
#             print(section)
#         section_type = indent
#         section_content = section_content[pointer:]

        if indent not in indent_dict:
            indent_dict[indent] = 1
        else:
            indent_dict[indent] += 1
            
        if indent <= fade_in_indent and prev_indent > fade_in_indent:
            new_sections.append(('DIALOG', section_name, section_texts))
            num_dialogs += 1
            section_name = 'NULL'
            section_texts = ''
            new_scene = True
            
#         print(sent)
        section_texts += re.sub(r'\t', '', sent).strip() + '\n'
        prev_indent = indent
        
    if new_scene:
        new_sections.append(('SCENE', section_name, section_texts))
        num_scenes += 1
    else:
        if prev_indent > fade_in_indent:
            new_sections.append(('DIALOG', section_name, section_texts))
            num_dialogs += 1
        else:
            new_sections.append(('SCENE', section_name, section_texts))
            num_scenes += 1
        
print(indent_dict)

# fileout = open('./preprocessed/with_bs/' + 'ff53fd53a94f343b8365915645b79d7ad5b1528e' + '.txt', 'w')
# for new_section in new_sections:
# #     print(new_section)
#     fileout.write(json.dumps(new_section))
#     fileout.write('\n')
# fileout.close()
print('num_scenes:', num_scenes)
print('num_dialogs:', num_dialogs)

['               ']
FADE IN indent: 15
{15: 1244, 25: 2178, 30: 230, 38: 1, 37: 1, 36: 1, 33: 1}
num_scenes: 476
num_dialogs: 952


In [120]:
#############################
##### Splitting scripts #####
#############################

count_movie = 0
count_meta_complete = 0
count_start_pattern = 0
count_summary = 0
    
if not os.path.exists('./imsdb_copy/bad_format/'):
    os.mkdir('./imsdb_copy/bad_format/')
# if not os.path.exists('./imsdb_copy/'):
#     os.mkdir('./imsdb_copy/')
    
count_with_fade_in = 0
list_num_scenes = []
list_num_dialogs = []

format_issue_dict = {}

num_verified = 0
num_failed = 0
num_movie = 0
num_fade_in_success = 0
    
for each_id in STORY_META.keys():
    if each_id in exclusive_id:
        format_issue_dict[each_id] = 'has some issues and will be skipped'
        print(each_id, 'has some issues and will be skipped')
        continue
        
    if each_id in special_id:
        format_issue_dict[each_id] = 'currently skipping the special script formats'
        print(each_id, 'currently skipping the special script formats')
        continue
        
#     each_id = 'cd13eb380843224f66c34047cc06dc445a92f8fd'
#     each_id = '18bcdf16fb896faf7fb36db6b752f16f0343ad24'
#     each_id = '1508452704b5829941b98aeee2eeba192f5beae5'
#     each_id = '00ee9e01a0e581e0d8cbf7e865a895147c480c5e'
    
    if each_id in good_script_dict:
        continue
       
    if os.path.isfile('./imsdb_copy/bad_format/{}.txt'.format(each_id)):
        continue
        
    num_movie += 1
    
    num_scenes = 0
    num_dialogs = 0
    
    with open(os.path.join('/Users/yisisang/Desktop/Character/imsdb_copy', each_id + '.txt'), 'r', encoding='cp1256') as file:
        content = file.read()

    soup = BeautifulSoup(content, 'html.parser')
    
    bs = soup.pre.find_all('b')

#     result = [[element.string, element.next_sibling.string] for element in bs if (element.next_sibling and element.next_sibling.string is not None)]

    sents = [(element.string, [ele for ele in element.next_sibling.string.split('\n') if ele.strip()]) #split the text by \n; remove space
             for element in bs if (element.next_sibling and element.next_sibling.string is not None)] #select text 
    
    if len(sents) < 200:
        format_issue_dict[each_id] = len(sents)
        print(each_id, ' ', len(sents))
    else:
        print('writing {} ...'.format(each_id))
#         print(sents)
        fileout = open(os.path.join('imsdb_copy/bad_format', each_id + '.txt'), 'w', encoding='cp1256')
        for sent in sents:
#             print(sent)
            if sent[0] is not None:
                section_name = re.sub(r'\t', '', sent[0]).strip()
            else:
                section_name = re.sub(r'\t', '', '\n').strip()
            section_texts = ''
            for line in sent[1]:
                section_texts += re.sub(r'\t', '', line).strip() + '\n'
            
            new_section = ('UNK', section_name, section_texts)
            
            fileout.write(json.dumps(new_section))
            fileout.write('\n')
        fileout.close()
        

writing Priest ...
writing Wanted ...
writing Enough ...
writing Revenant-The ...
writing Dawn-of-the-Dead ...
Real-Genius   0
Conan-the-Barbarian   0
Blade-II   0
writing Gremlins-2 ...
writing Breakfast-Club-The ...
writing Speed-Racer ...
writing Piano-The ...
writing Mute-Witness ...
Copycat   0
writing Alien-vs-Predator ...
writing Titanic ...
writing Fighter-The ...
writing Dogma ...
writing In-the-Bedroom ...
writing John-Wick ...
Resident-Evil   0
writing Barry-Lyndon ...
writing No-Strings-Attached ...
writing Shrek ...
Platoon   0
writing Anniversary-Party-The ...
Brazil   0
Apartment-The   2
Crow-Salvation-The   2
writing Happy-Birthday-Wanda-June ...
Never-Been-Kissed   0
writing Dances-with-Wolves ...
writing Coriolanus ...
Air-Force-One   1
Almost-Famous   2
Youve-Got-Mail   2
Commando   0
writing Red-Riding-Hood ...
writing Pet-Sematary-II ...
Planet-of-the-Apes-The   0
writing Mimic ...
Punch-Drunk-Love   2
writing Ni-vu-ni-connu ...
Labyrinth   172
writing Unknown ...


writing Road-The ...
Who-Framed-Roger-Rabbit   0
writing Good-Girl-The ...
writing Thor ...
writing Star-Wars-Return-of-the-Jedi ...
Halloween-The-Curse-of-Michael-Myers   0
writing Deception ...
writing Serenity ...
Training-Day   0
writing Blue-Valentine ...
writing Drive ...
writing Margaret ...
writing Knocked-Up ...
writing Watchmen ...
writing Two-For-The-Money ...
writing Clash-of-the-Titans ...
writing Burlesque ...
writing His-Girl-Friday ...
writing Crash ...
writing All-About-Steve ...
writing Surrogates ...
writing Silver-Linings-Playbook ...


    0025577043f5090cd603c6aea60f26e236195594   0
    069ba0ceb78ef8fe8a193b7b4f225938f2044f0c   0
    08172f313a4053b150a6c20310b6ee58f6968b76 currently skipping the special script formats
    0bc7352d6a0e678c0d8acc57c0c1cc3466fe9ef7 has some issues and will be skipped
    1aae28477e771b3af008ec59ce29086a1bc66776 has some issues and will be skipped
    2e0ecbcbf69543a8d94e5391488ff4875174da74   172
    35592c2abea624d315c5171d67ab5e14794ca071   0
    3747036f950fe8f79cdaa0eb713104b9eb8af6c5 has some issues and will be skipped
    37c11f984cb14401d85abfc20e8305ca7a472c9f currently skipping the special script formats
    39b1f44ec2639226b6025140e2a7e7eaf570409e has some issues and will be skipped
    405df1ccf0409ea6040c2a765a0315878f991d79 has some issues and will be skipped
    518bfcb6ea413715080cbd6f97aa479ac5f6c7d3   0
    5283fa0a6ea69f4b4224d12018bbf985a2b80543 has some issues and will be skipped
    54e7219964513efc449b07327b717c600c609115   6
    5a284f4743e8864970dde010c6f0ce08fa787090   0
    69add8ffcf964d23243ac02b7282d57beb63c355   176
    823a8b6bbbb0bbeec9c4bec1929b64a3694232d7 has some issues and will be skipped
    90c5a4e276e3f9dec41d7be579052b9327f56197   7
    91075ab5383e8113d86b17536c6918bfbbd6af20   16
    935775d945bc210b0928bffb6924e06e1ef9a9dd   1
    a1f45ca7bfe2e685b4faa4032039a3a5079065ac   0
    b5aacdd9eb15f4b978fd7d841d62c1167ba1b68b   0
    c085c72c78f6ad35854654d222a54b016b08dc65   33
    d053fe3c068b8a68d07d7384056afd16935e608a   75
    db25c1e66223c9829dbec103c7e4b1820d240c08   0
    de42ec88ce00d0f1e21ad4ad719d1d64499e5166   3
    fa9531b58c04a11f5d31e87cc73e797f58d6894c   7
    fba51a67cf7aa08f14b84721571e67cc840d8203   0

In [ ]:
failed_id_dict = [
    '1aae28477e771b3af008ec59ce29086a1bc66776',     # download failed?
    '3747036f950fe8f79cdaa0eb713104b9eb8af6c5',     # download failed?
    '5283fa0a6ea69f4b4224d12018bbf985a2b80543',     # download failed?
]


count_movie = 0
count_meta_complete = 0
count_start_pattern = 0
count_summary = 0
if not os.path.exists('./preprocessed/'):
    os.mkdir('./preprocessed/')
if not os.path.exists('./preprocessed/movies/'):
    os.mkdir('./preprocessed/movies/')
    
for each_id in STORY_META.keys():
    if each_id not in format_issue_dict:
        continue
        
    if each_id in failed_id_dict:
        print(each_id, 'download failed')
        continue
#     each_id = '37c11f984cb14401d85abfc20e8305ca7a472c9f'
#     print(each_id)
    
    if each_id == '37c11f984cb14401d85abfc20e8305ca7a472c9f':
        fp = open(os.path.join('tmp', each_id + '.content'),'r', encoding='utf16')
    else:
        fp = open(os.path.join('tmp', each_id + '.content'),'r', encoding='cp1256')
        
#         fileout = open('./preprocessed/movies/' + each_id + '.json', 'w')
    this_movie          = dict()
    this_movie['content'] = list()
    content             = ''
    line                = ' '
    empty_line_in_row   = 0
    meta_prepared       = False
    start_pattern       = False
    pre_tag_line_idxes  = list()
    _pre_tag_line_idxes = list()
    concat_string       = ''
    concat_start        = False
    concat_end          = False

    lines = list()
    while line:
        try:
            line = fp.readline()
        except:
            line = ' '
            continue
        new_line = re.sub(r'\t', '', line).strip()
#             new_line = re.sub(r'\t', '    ', line).strip('\n')

        new_line_in_low = new_line.lower()
        lines.append(new_line)

#             print(new_line)
#             if 'body' in new_line:
#                 print(new_line)

        # if it is the start of meta data
        if len(lines) < 20 and not meta_prepared and \
            ('<title>' in new_line and '</title>' in new_line or '<head><title>' in new_line):
            meta_start_idx  = new_line.find('<title>') + len('<title>')
            meta_end_idx    = new_line.find('Script')
            meta_end_idx    = meta_end_idx if meta_end_idx > 0 else (len(new_line) + 5)
            this_movie['title'] = new_line[meta_start_idx : meta_end_idx].strip()
            cells           = this_movie['title'].split(',')
            if len(cells) == 1:
                this_movie['title'] = cells[0]
            else:
                if 'script' in cells[-1].lower() or 'imsdb' in cells[-1].lower():
                    this_movie['title'] = ','.join(cells[:-1])
                else:
                    this_movie['title'] = ','.join(cells)
            meta_prepared   = True
            count_meta_complete += 1
        # if meta data is ready

        if meta_prepared:
            if '<pre>' in new_line:
                pre_tag_line_idxes.append(len(lines)-1)
            if '</pre>' in new_line:
                _pre_tag_line_idxes.append(len(lines))

#     print(lines)
#     print(pre_tag_line_idxes, _pre_tag_line_idxes)
    # use <pre> </pre> pairs to crop drama content
    if len(pre_tag_line_idxes) < 3:
        lines = lines[pre_tag_line_idxes[-1] : _pre_tag_line_idxes[0]]
    elif len(pre_tag_line_idxes) == 6:
        lines = lines[pre_tag_line_idxes[1] : _pre_tag_line_idxes[0]]
    elif len(pre_tag_line_idxes) == 8:
        lines = lines[pre_tag_line_idxes[1] : _pre_tag_line_idxes[6]]
    else:
        print(each_id, 'has <pre> tag issue and will be skipped.')
        continue
    # sometimes the drama has format issue
    if len(lines) > 700:
        count_start_pattern += 1
    else:
        print(each_id, 'has content-format issue and will be skipped.')
        continue

#         print(lines)
    content = '\n'.join(lines)
    sections = content.split('\n\n')

    if len(sections) < 500:
        print(each_id, len(sections))
    else:
        print('writing {} ...'.format(each_id))
        fileout = open(os.path.join('preprocessed/format_problems', each_id + '.txt'), 'w', encoding='cp1256')
        
        for section in sections:
            sents = section.split('\n')
            
            section_name = re.sub(r'\t', '', sents[0]).strip()

            section_texts = ''
            for sent in sents[1:]:
                section_texts += re.sub(r'\t', '', line).strip() + '\n'
            
            new_section = ('UNK', section_name, section_texts)
            
            fileout.write(json.dumps(new_section))
            fileout.write('\n')
        fileout.close()
#     break

In [116]:
print(len(format_issue_dict))

filenames = os.listdir('imsdb_copy/silver') + os.listdir('imsdb_copy/by_stats')
print(len(filenames))

good_script_dict = {}
for filename in filenames:
    filename = filename.replace('.txt', '')
    good_script_dict[filename] = 1


0
789


In [117]:
print(len(format_issue_dict))

0
